In [1]:
import torch
import numpy as np
from skimage import io, transform
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from preprocess import GetDataset
import random

/Users/eriq/opt/anaconda3/envs/pix2pix-pytorch/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/eriq/opt/anaconda3/envs/pix2pix-pytorch/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <DDABACEB-F2EA-368C-80DD-40745DFB96F8> /Users/eriq/opt/anaconda3/envs/pix2pix-pytorch/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <9DD59EBB-EC3F-3DDE-B60A-BC415663D633> /Users/eriq/opt/anaconda3/envs/pix2pix-pytorch/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [5]:
# Optimizers
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=2e-3, betas=(0.5, 0.999))
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=2e-3, betas=(0.5, 0.999))


In [228]:
dataset = GetDataset()


OUTPUT_CHANNELS = 3

import torch.torch.nn.functional as F
import math

def get_padding(size, kernel_size, stride, dilation):
    padding = ((size - 1) * (stride - 1) + dilation * (kernel_size - 1)) //2
    return padding
    
def downsample(in_channels, out_channels, size, apply_batchnorm=True):
    #3 in_channels, 3 out_channels, 4x4 size, 2 stride
    padding = get_padding(256, size, 2, 1)
    print(padding)
    conv = torch.torch.torch.torch.torch.torch.torch.torch.nn.Conv2d(in_channels, out_channels, size, 2, padding) 
    torch.torch.nn.init.normal(conv.weight, 0, 0.02)
    batchnorm = torch.torch.nn.BatchNorm2d(out_channels) 
    result = torch.torch.nn.Sequential()
    result.append(conv)
    if apply_batchnorm:
        result.append(batchnorm)
    result.append(torch.torch.nn.LeakyReLU())
    result.eval()
    return result

el, sat = convert_ds_to_tensor(dataset)
inputs = torch.tensor(el[40:41])
down_model = downsample(3, 3, 4)
down_result = down_model(inputs)
print (down_result.shape)



129
torch.Size([1, 3, 256, 256])


/var/folders/3g/2wscfy492xv8p55yn6b0rthw0000gn/T/ipykernel_53719/230000692.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(el[40:41])
/var/folders/3g/2wscfy492xv8p55yn6b0rthw0000gn/T/ipykernel_53719/230000692.py:18: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(conv.weight, 0, 0.02)


In [225]:

    
def upsample(in_channels, out_channels, size, apply_dropout=False):
    #3 in_channels, 3 out_channels, 4x4 size, 2 stride
    convT = torch.torch.nn.ConvTranspose2d(in_channels, out_channels, size, stride=2, bias=False)
    torch.torch.nn.init.normal(convT.weight, 0, 0.02)
    # No batchnorm if out_channel is of size 1
    batchnorm = torch.torch.nn.BatchNorm2d(out_channels) 
    relu = torch.torch.nn.ReLU()
    dropout = torch.torch.nn.Dropout()
    result = torch.torch.nn.Sequential()
    result.append(convT)
    result.append(batchnorm)
    if apply_dropout:
        result.append(dropout)
    result.append(relu)
    
    return result


In [226]:

#inputs go here
loss_object = torch.torch.nn.BCEWithLogitsLoss()

# down_model = downsample(None)
# up_model = upsample(None)

class Generator(torch.torch.nn.Module):

    def __init__(self):
        super().__init__()

        # inputs = torch.tensor(np.zeros(3,256,256))
        self.down_stack = [
            downsample(3, 64, 4, apply_batchnorm=False),  # (batch_size, 128, 128, 64)
            downsample(64, 128, 4),  # (batch_size, 64, 64, 128)
            downsample(128, 256, 4),  # (batch_size, 32, 32, 256)
            downsample(256, 512, 4),  # (batch_size, 16, 16, 512)
            downsample(512, 512, 4),  # (batch_size, 8, 8, 512)
            downsample(512, 512, 4),  # (batch_size, 4, 4, 512)
            downsample(512, 512, 4),  # (batch_size, 2, 2, 512)
            downsample(512, 512, 4),  # (batch_size, 1, 1, 512)
        ]

        self.up_stack = [
            upsample(512, 512, 2, apply_dropout=True),  # (batch_size, 2, 2, 1024)
            upsample(1024, 512, 2, apply_dropout=True),  # (batch_size, 4, 4, 1024)
            upsample(1024, 512, 2, apply_dropout=True),  # (batch_size, 8, 8, 1024)
            upsample(1024, 512, 2),  # (batch_size, 16, 16, 1024)
            upsample(1024, 256, 2),  # (batch_size, 32, 32, 512)
            upsample(512, 128, 2),  # (batch_size, 64, 64, 256)
            upsample(256, 64, 2),  # (batch_size, 128, 128, 128)
        ]
        
        self.last = torch.torch.nn.ConvTranspose2d(128, 3, 2, stride=2, bias=False)
        self.tanh = torch.torch.nn.Tanh()
        torch.torch.nn.init.normal(self.last.weight, 0, 0.02)

    def forward(self, x):
        # Downsampling through the model
        skips = []
        for down in self.down_stack:
            print(x.shape)
            x = down(x)
            skips.append(x)

        skips = reversed(skips[:-1])

        # Upsampling and establishing the skip connections
        for up, skip in zip(self.up_stack, skips):
            print(x.shape)
            x = up(x)
            # Concat skip with upsampled along channels dimension
            x = torch.cat((x,skip), 1)
        x = self.last(x)
        x = self.tanh(x)

        return x

generator = Generator()

def convert_ds_to_tensor(ds):
    elevation_imgs = []
    satellite_imgs = []
    for i in range(len(ds)):
        elevation_imgs.append(ds[i]['elevation'])
        satellite_imgs.append(ds[i]['satellite'])
    
    elevation_imgs = torch.stack(elevation_imgs)
    satellite_imgs = torch.stack(satellite_imgs)

    return elevation_imgs, satellite_imgs
el, sat = convert_ds_to_tensor(dataset)
inputs = torch.tensor(el[40:41])
#print(inputs[0])
#plt.imshow(np.transpose(inputs[0].detach().numpy(), (1,2,0)))
gen = generator(torch.tensor(inputs.detach().numpy()))
plt.imshow(np.transpose(gen[0].detach().numpy(), (1,2,0)) * 0.5 + 0.5)
print(gen[0])

ValueError: padding='same' is not supported for strided convolutions

In [230]:
from torch import nn

class UnetSkipConnectionBlock(nn.Module):
    """Defines the Unet submodule with skip connection.
        X -------------------identity----------------------
        |-- downsampling -- |submodule| -- upsampling --|
    """

    def __init__(self, outer_nc, inner_nc, input_nc=None,
                 submodule=None, outermost=False, innermost=False, norm_layer=nn.BatchNorm2d, use_dropout=False):
        """Construct a Unet submodule with skip connections.
        Parameters:
            outer_nc (int) -- the number of filters in the outer conv layer
            inner_nc (int) -- the number of filters in the inner conv layer
            input_nc (int) -- the number of channels in input images/features
            submodule (UnetSkipConnectionBlock) -- previously defined submodules
            outermost (bool)    -- if this module is the outermost module
            innermost (bool)    -- if this module is the innermost module
            norm_layer          -- normalization layer
            use_dropout (bool)  -- if use dropout layers.
        """
        super(UnetSkipConnectionBlock, self).__init__()
        self.outermost = outermost
        if type(norm_layer) == functools.partial:
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d
        if input_nc is None:
            input_nc = outer_nc
        downconv = nn.Conv2d(input_nc, inner_nc, kernel_size=4,
                             stride=2, padding=1, bias=use_bias)
        downrelu = nn.LeakyReLU(0.2, True)
        downnorm = norm_layer(inner_nc)
        uprelu = nn.ReLU(True)
        upnorm = norm_layer(outer_nc)

        if outermost:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1)
            down = [downconv]
            up = [uprelu, upconv, nn.Tanh()]
            model = down + [submodule] + up
        elif innermost:
            upconv = nn.ConvTranspose2d(inner_nc, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1, bias=use_bias)
            down = [downrelu, downconv]
            up = [uprelu, upconv, upnorm]
            model = down + up
        else:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1, bias=use_bias)
            down = [downrelu, downconv, downnorm]
            up = [uprelu, upconv, upnorm]

            if use_dropout:
                model = down + [submodule] + up + [nn.Dropout(0.5)]
            else:
                model = down + [submodule] + up

        self.model = nn.Sequential(*model)
        def forward(self, x):
            if self.outermost:
                return self.model(x)
            else:   # add skip connections
                return torch.cat([x, self.model(x)], 1)

In [231]:
class UnetGenerator(nn.Module):
    """Create a Unet-based generator"""

    def __init__(self, input_nc, output_nc, num_downs, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=False):
        """Construct a Unet generator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            output_nc (int) -- the number of channels in output images
            num_downs (int) -- the number of downsamplings in UNet. For example, # if |num_downs| == 7,
                                image of size 128x128 will become of size 1x1 # at the bottleneck
            ngf (int)       -- the number of filters in the last conv layer
            norm_layer      -- normalization layer
        We construct the U-Net from the innermost layer to the outermost layer.
        It is a recursive process.
        """
        super(UnetGenerator, self).__init__()
        # construct unet structure
        unet_block = UnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=None, norm_layer=norm_layer, innermost=True)  # add the innermost layer
        for i in range(num_downs - 5):          # add intermediate layers with ngf * 8 filters
            unet_block = UnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer, use_dropout=use_dropout)
        # gradually reduce the number of filters from ngf * 8 to ngf
        unet_block = UnetSkipConnectionBlock(ngf * 4, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(ngf * 2, ngf * 4, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(ngf, ngf * 2, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        self.model = UnetSkipConnectionBlock(output_nc, ngf, input_nc=input_nc, submodule=unet_block, outermost=True, norm_layer=norm_layer)  # add the outermost layer

    def forward(self, input):
        """Standard forward"""
        return self.model(input)

In [233]:
import functools

def define_g(ngf, use_dropout=False, init_type='normal', init_gain=0.02, gpu_ids=[]):
    norm_layer = functools.partial(nn.InstanceNorm2d, affine=False, track_running_stats=False)
    net = UnetGenerator(3, 3, 8, ngf, norm_layer=norm_layer, use_dropout=use_dropout)
    init_weights(net, init_type, init_gain=init_gain)
    return net




In [ ]:
    

def generator_loss(disc_generated_output, gen_output, target):
    gan_loss = loss_object(disc_generated_output, torch.ones_like(disc_generated_output))

    # mean absolute error
    l1_loss = torch.mean(torch.abs(target - gen_output))

    total_gen_loss = gan_loss + (10 * l1_loss)

    return total_gen_loss, gan_loss, l1_loss

In [146]:

discriminator = Discriminator()

# Training
def train_step(input_images, targets, batch_size=1):

    num_imgs = len(input_images)
    num_batches = int(num_imgs / batch_size)

    total_gen_loss = total_gan_loss = total_l1_loss = total_disc_loss = total_seen = 0

    for index, end in enumerate(range(batch_size, num_imgs+1, batch_size)):
        start = end - batch_size

        input_batch = input_images[start:end]
        target_batch = targets[start:end]        

        gen_output = generator(input_batch) # self.forward 

        for param in discriminator.parameters():
            param.requires_grad = True

        discriminator_optimizer.zero_grad() #update D gradients
        disc_generated_output = discriminator(input_batch, gen_output.detach())
        disc_gen_loss = discriminator_generated_loss(disc_generated_output)
        disc_real_output = discriminator(input_batch, target_batch) 
        disc_real_loss = discriminator_real_loss(disc_real_output)
        disc_loss = (disc_gen_loss + disc_real_loss) * 0.5
        disc_loss.backward()
        discriminator_optimizer.step()
        
        for param in discriminator.parameters():
             param.requires_grad = False

        generator_optimizer.zero_grad() #update G gradients
        disc_generated_output = discriminator(input_batch, gen_output)     
        gen_loss, gan_loss, l1_loss = generator_loss(disc_generated_output, gen_output, target_batch)
        disc_generated_output = discriminator(input_batch, gen_output)

        gen_loss.backward()
        generator_optimizer.step()

        total_seen += batch_size

        total_gen_loss += gen_loss
        total_gan_loss += gan_loss
        total_l1_loss += l1_loss
        total_disc_loss += disc_loss

        avg_gen_loss = float(total_gen_loss / total_seen)
        avg_gan_loss = float(total_gan_loss / total_seen)
        avg_l1_loss = float(total_l1_loss / total_seen)
        avg_disc_loss = float(total_disc_loss / total_seen)

        print(f"\r[Batch {index+1}/{num_batches}]\t gen_loss={avg_gen_loss:.3f}\t disc_loss: {avg_disc_loss:.3f}", end='')
    
    return avg_gen_loss, avg_gan_loss, avg_l1_loss, avg_disc_loss

def fit(train_ds, test_ds, epochs):
    train_elevation_imgs, train_satellite_imgs = convert_ds_to_tensor(train_ds)
    test_elevation_imgs, test_satellite_imgs = convert_ds_to_tensor(test_ds)
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        train_step(train_elevation_imgs, train_satellite_imgs)
        generate_images()
        if epoch % 5 + 1 == 0:
            generate_images()

/var/folders/3g/2wscfy492xv8p55yn6b0rthw0000gn/T/ipykernel_53719/2418674756.py:9: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(conv.weight, 0, 0.02)


In [150]:

# Generate images
def generate_images(model, test_input, tar):
    prediction = model(test_input)
    plt.figure(figsize=(15, 15))

    i = np.random.randint(0, test_input.shape[0])

    input_img = np.moveaxis(test_input[i].detach().numpy(), 0, 2) 
    ground_img = np.moveaxis(tar[i].detach().numpy(), 0, 2)
    pred_img = np.moveaxis(prediction[i].detach().numpy(), 0, 2)
    print(pred_img)

    display_list = [input_img, ground_img, pred_img]
    title = ['Input Image', 'Ground Truth', 'Predicted Image']

    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        # Getting the pixel values in the [0, 1] range to plot.
        if (title[i] == "Predicted Image"):
            plt.imshow(display_list[i] * 0.5 + 0.5)
        else:
            plt.imshow(display_list[i])
        plt.axis('off')
    plt.show()

In [148]:
if __name__ == '__main__':
    print('<-----------------Runnin----------------->')
    elevation_imgs, satellite_imgs = convert_ds_to_tensor(dataset)

    fit(dataset, dataset, 1)

    torch.save(generator.state_dict(), 'generator.pth')
    torch.save(discriminator.state_dict(), 'discriminator.pth')

    elevation_imgs, satellite_imgs = convert_ds_to_tensor(dataset)


<-----------------Runnin----------------->

Epoch 1/1


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 512, 1, 1])

In [87]:
inputs = elevation_imgs[7:10]
im = np.transpose(generator(inputs)[0].detach().numpy(), (1,2,0))
#should be ranging from [-1, 1]
print(im * 0.5 + 0.5)

#100 epochs, 1000 images --> 100,000 images processed over 10 hours --> 0.36 seconds per

def generate_images():
    inputs = elevation_imgs[5:6]
    plt.imshow(np.transpose(inputs[0].detach().numpy(), (1,2,0)) * 255) 

[[[0.5170993  0.4953152  0.49022812]
  [0.47941688 0.49018908 0.47456288]
  [0.5292173  0.48597473 0.49127313]
  ...
  [0.50379443 0.49938473 0.45683804]
  [0.5233289  0.49753067 0.48497382]
  [0.49837086 0.49592152 0.47976297]]

 [[0.45694968 0.49943554 0.49275032]
  [0.49206924 0.51005703 0.50993836]
  [0.46452558 0.49468717 0.48920575]
  ...
  [0.49631798 0.5046685  0.52099   ]
  [0.4754678  0.50714016 0.50419796]
  [0.5281023  0.50035036 0.533948  ]]

 [[0.5165292  0.52366215 0.48499838]
  [0.4615075  0.4709982  0.45580754]
  [0.5342196  0.50022197 0.47996566]
  ...
  [0.49300197 0.4708811  0.44118258]
  [0.51905537 0.49788648 0.48586836]
  [0.5066682  0.46902347 0.4770921 ]]

 ...

 [[0.49198523 0.50787306 0.50888383]
  [0.46178868 0.4739791  0.5021978 ]
  [0.47884962 0.5030491  0.48750862]
  ...
  [0.46538794 0.460155   0.49641216]
  [0.4877821  0.5107234  0.4959717 ]
  [0.5119074  0.4632984  0.54073226]]

 [[0.4888115  0.5278495  0.48331448]
  [0.4534592  0.5156974  0.4496961 ]
